# Tools and Routing

In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
from langchain.agents import tool

In [ ]:
@tool # converts the function into a langchain tool
def search(query: str) -> str:
    """Search for weather online"""
    return "42f"

search.name, search.description, search.args

In [ ]:
# Define a structure for the input schema
# Important since the LLM use it as a prompt
from pydantic import BaseModel, Field

class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")

@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for weather online"""
    return "42f"

search.name, search.description, search.args

In [ ]:
search.run("CDG")

## Application 1: Get temperature given a location latitude and longitude

In [ ]:
import requests
from datetime import datetime

# Input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(description="Latitude of the location to fetch weather data")
    longitude: float = Field(description="Longitude of the location to fetch weather data")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""

    BASE_URL = "https://api.open-meteo.com/v1/forecast"

    # Parameters for the request
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m",
        "forecast_days": 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.now()
    time_list = [datetime.fromisoformat(time_str.replace("Z", "+00:00")) for time_str in results["hourly"]["time"]]
    temperature_list = results["hourly"]["temperature_2m"]
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f"{current_temperature}°C"

In [ ]:
get_current_temperature.name, get_current_temperature.description, get_current_temperature.args

In [ ]:
# Convert the tool to an actual OpenAI function
from langchain_core.utils.function_calling import convert_to_openai_function
get_current_temperature_openai = convert_to_openai_function(get_current_temperature)

In [ ]:
from langchain_openai import ChatOpenAI
import json

# Create the request and ask the model to identify the function inputs
latitude = 3.87
longitude = 11.52

ask_weather = """What's the current temperature in:
latitude = {latitude}
longitude = {longitude}
"""

messages = [
    {"role": "user", "content": ask_weather.format(latitude=latitude, longitude=longitude)},
]

model = ChatOpenAI().bind(functions=[get_current_temperature_openai])
response = model.invoke(messages)
messages.append(response)

# Retrieve the inputs and execute the function
args = json.loads(response.additional_kwargs["function_call"]["arguments"])
temperature = get_current_temperature.run(args)

# Add the function output to the conversation and display the final response
messages.append({
    "role": "function",
    "name": response.additional_kwargs["function_call"]["name"],
    "content": temperature,
})

response = model.invoke(messages)
response.content

## Application 2: Summarise Wikipedia pages

In [ ]:
import wikipedia

@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia Search and get page summaries"""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[:3]:
        try:
            wiki_page = wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DIsambiguationError,
        ):
            pass
    
    if not summaries:
        return "No good Wikipedia Search Result was found"

    return "".join(summaries)

In [ ]:
search_wikipedia.name, search_wikipedia.description, search_wikipedia.args

In [ ]:
result = search_wikipedia.invoke({"query": "langchain"})
print(result)

## Create OpenAI function from an OpenAPI Specification (OAS) 

The OAS defines a standard, language-agnostic interface to HTTP APIs which allows both humans and computers to discover and understand the capabilities of the service without access to source code.

In [ ]:
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
from langchain.utilities.openapi import OpenAPISpec

In [ ]:
text = """
{
  "openapi": "3.1.0",
  "info": {
    "version": "1.0.0",
    "title": "Swagger Petstore",
    "license": {
      "name": "MIT"
    }
  },
  "servers": [
    {
      "url": "http://petstore.swagger.io/v1"
    }
  ],
  "paths": {
    "/pets": {
      "get": {
        "summary": "List all pets",
        "operationId": "listPets",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "limit",
            "in": "query",
            "description": "How many items to return at one time (max 100)",
            "required": false,
            "schema": {
              "type": "integer",
              "maximum": 100,
              "format": "int32"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "A paged array of pets",
            "headers": {
              "x-next": {
                "description": "A link to the next page of responses",
                "schema": {
                  "type": "string"
                }
              }
            },
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pets"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      },
      "post": {
        "summary": "Create a pet",
        "operationId": "createPets",
        "tags": [
          "pets"
        ],
        "responses": {
          "201": {
            "description": "Null response"
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    },
    "/pets/{petId}": {
      "get": {
        "summary": "Info for a specific pet",
        "operationId": "showPetById",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "petId",
            "in": "path",
            "required": true,
            "description": "The id of the pet to retrieve",
            "schema": {
              "type": "string"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pet"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "Pet": {
        "type": "object",
        "required": [
          "id",
          "name"
        ],
        "properties": {
          "id": {
            "type": "integer",
            "format": "int64"
          },
          "name": {
            "type": "string"
          },
          "tag": {
            "type": "string"
          }
        }
      },
      "Pets": {
        "type": "array",
        "maxItems": 100,
        "items": {
          "$ref": "#/components/schemas/Pet"
        }
      },
      "Error": {
        "type": "object",
        "required": [
          "code",
          "message"
        ],
        "properties": {
          "code": {
            "type": "integer",
            "format": "int32"
          },
          "message": {
            "type": "string"
          }
        }
      }
    }
  }
}
"""

In [ ]:
spec = OpenAPISpec.from_text(text)
pet_openai_functions, pet_callables = openapi_spec_to_openai_fn(spec)

In [ ]:
pet_openai_functions

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0).bind(functions=pet_openai_functions)
model.batch([
    "What are three pets names?",
    "Tell me about the pet with id 21",
])